In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import notebook_helper
import mprompt.viz
import scipy.special
from collections import defaultdict
from pprint import pprint
import joblib
import viz
from mprompt.config import RESULTS_DIR


# EXPT_NAME = 'relationships_mar9'
# EXPT_NAME = 'huth2016clusters_mar21_i_time_traveled'
# EXPT_NAME = 'voxels_mar21_hands_arms_emergency'
seed = 2
EXPT_NAME = f'uts02_concepts_pilot_mar22_seed={seed}'
EXPT_DIR = join(RESULTS_DIR, 'stories', EXPT_NAME)
EXPT_DIRS = [join(RESULTS_DIR, 'stories', f'uts02_concepts_pilot_mar22_seed={seed}')
             for seed in range(1, 8)]
rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
expls = rows.expl.values
paragraphs = rows.paragraph.values
prompts = rows.prompt.values
# voxel_nums = rows.module_num.values
# subjects = rows.subject.values

### Heatmaps

**Explanation** <> Story Heatmap

In [ ]:
for EXPT_DIR in EXPT_DIRS:
    scores_data = joblib.load(join(EXPT_DIR, f'scores_data.pkl'))
    s = scores_data['scores_mean'].T
    # s = scipy.special.softmax(s, axis=1)
    # s = (s - s.min()) / (s.max() - s.min())
    rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
    expls = rows.expl.values
    mprompt.viz.heatmap(s, expls, ylab='Story', xlab='Explanation')
    # plt.savefig(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_story_data_match.png'), dpi=300)
    # plt.savefig(join(RESULTS_DIR, 'stories', f'{EXPT_NAME}_story_data_match.pdf'), bbox_inches='tight')
    # plt.tight_layout()
    # plt.show()
viz.save_figs_to_single_pdf('../results/expl_story_match.pdf')

**Module** <> Story Heatmap

In [ ]:
for EXPT_DIR in EXPT_DIRS:
    # keys: scores_mod, scores_max_mod, all_scores, all_ngrams
    # ngram_lengths = [10, 50, 100, 384]
    ngram_lengths = [0]
    for ngram_length in ngram_lengths:
        scores_mod_dict = joblib.load(join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
        rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
        expls = rows.expl.values

        s = scores_mod_dict['scores_mean'].T
        s = scipy.special.softmax(s, axis=0)
        # s = (s - s.min()) / (s.max() - s.min())
        mprompt.viz.heatmap(s, expls, ylab='Story', xlab='Module (fit to fMRI voxel)', clab='Mean module response')

        n = s.shape[0]
        # plt.plot([0, n], [0, n], '--', color='gray', alpha=0.1)
        plt.title(f'Model window: {ngram_length} words', fontsize='small')
        plt.savefig(join(EXPT_DIR, f'{EXPT_NAME}_story_mod_match_ngram_length={ngram_length}.png'), dpi=300)
        plt.savefig(join(EXPT_DIR, f'{EXPT_NAME}_story_mod_match_ngram_length={ngram_length}.png'), bbox_inches='tight')
        # plt.show()
viz.save_figs_to_single_pdf('../results/expl_mod_match.pdf')

### Best voxels automated

In [10]:
vox_diffs = defaultdict(list)
for EXPT_DIR in EXPT_DIRS:
    ngram_length = 0
    scores_mod_dict = joblib.load(join(EXPT_DIR, f'scores_mod_ngram_length={ngram_length}.pkl'))
    rows = joblib.load(join(EXPT_DIR, f'rows.pkl'))
    expls = rows.expl.values


    s = scores_mod_dict['scores_mean'].T
    s = scipy.special.softmax(s, axis=0)

    BLOCK_SIZE = 4
    for i in range(0, rows.shape[0]):
        row = rows.iloc[i]
        s_row = s[i]
        block_idxs = range((i // BLOCK_SIZE) * BLOCK_SIZE, (i // BLOCK_SIZE + 1) * BLOCK_SIZE)
        diff = s_row[i] - np.mean([s_row[j] for j in range(s_row.shape[0]) if j not in block_idxs])
        vox_diffs[(row.module_num, row.expl)].append(diff)
vox_diffs = {k: np.mean(v) for k, v in vox_diffs.items()}

In [15]:
# display all rows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(vox_diffs.items()).sort_values(1, ascending=False).head(30))

,0,1
92,"(337, moment)",0.003091
49,"(122, locations)",0.002656
25,"(168, surprise)",0.002644
64,"(171, measurements)",0.002610
66,"(368, locations)",0.002441
47,"(79, food preparation)",0.002327
76,"(398, emotional expression)",0.002319
78,"(299, communication)",0.001988
14,"(47, communication)",0.001979
60,"(212, time)",0.001943
